In [0]:
from io import StringIO
import pandas as pd

In [0]:
%run /Workspaces/dars_nic_391419_j3w9t_collab/CCU013_02/atlas/00_Load_helpers

### primary_care medications table:

In [0]:
%sql 
select * from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t limit 5;

In [0]:
%sql 
select count(distinct(PrescribedBNFCode)) from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t;

In [0]:
%sql 
select count(*) from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t
where ProcessingPeriodDate between to_date('2019-03-01', 'yyyy-MM-dd') and to_date('2020-02-01', 'yyyy-MM-dd');

In [0]:
%sql 
select max(ProcessingPeriodDate) from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t;

**Prescription events for cohort and relevant time**

assuming 1 month delay (prescriptions issued in a month are submitted to nhsbsa in the following month and appear in the table as processing date of 01-XX-YYYY)

We capture jan 2019 to jan 2020 with: 02-02-2019 (prescription issued in jan 2019) to 02-02-2020 (prescriptions issued in jan 2020)

In [0]:
%sql 
select * from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t limit 5;

In [0]:
ccu013_02_prim_care_meds_cohort = spark.sql('''
select m.Person_ID_DEID, m.PrescribedBNFCode, m.ProcessingPeriodDate, c.age_band_23_01_2020, c.sex from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t m 
inner join dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c 
on m.person_ID_DEID = c.NHS_NUMBER_DEID
where m.ProcessingPeriodDate between to_date('2019-03-01', 'yyyy-MM-dd') and to_date('2020-02-01', 'yyyy-MM-dd')
and c.SEX IN (1,2) 
AND c.IMD_quintile not like ('Unknown%') 
AND c.age_23_01_2020 >= 0 and c.age_23_01_2020 <= 100''')


In [0]:
ccu013_02_prim_care_meds_cohort = ccu013_02_prim_care_meds_cohort.withColumn("BNF6", ccu013_02_prim_care_meds_cohort.PrescribedBNFCode.substr(1,6))
ccu013_02_prim_care_meds_cohort.show()

In [0]:
ccu013_02_prim_care_meds_cohort.createOrReplaceGlobalTempView('ccu013_02_prim_care_meds_cohort') 

In [0]:
drop_table("ccu013_02_prim_care_meds_cohort")

In [0]:
create_table("ccu013_02_prim_care_meds_cohort")

In [0]:
  %sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort limit 20;

In [0]:
%sql 
select count(distinct(person_ID_DEID)) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort limit;

In [0]:
%sql 
select count(distinct(PrescribedBNFCode)) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort;

In [0]:
%sql 
select count(distinct(ProcessingPeriodDate)) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort;

In [0]:
%sql 
select distinct(ProcessingPeriodDate) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort;

In [0]:
%sql 
select count(distinct(BNF6)) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort;

In [0]:
%sql 
select distinct(BNF6)  from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort;

In [0]:
%sql 
select max(ProcessingPeriodDate) from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t;

In [0]:
%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort where BNF6 == '200202';

In [0]:
%sql 
select BNF6, count(distinct(Person_ID_DEID)) 
from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort where BNF6= '050101'
group by BNF6;

**Select first prescription only (use this table in the analysis)**

In [0]:
%sql 
select m.Person_ID_DEID, 
  m.PrescribedBNFCode, 
  min(m.ProcessingPeriodDate) as first_prescrip_event_in_the_year, 
  m.age_band_23_01_2020, 
  m.sex, 
  m.BNF6  
from 
  dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort m 
group by m.Person_ID_DEID, m.PrescribedBNFCode, m.age_band_23_01_2020, m.sex, m.BNF6;

first event (first prescription only) - we look at unique groups regardless of how many prescriptions in this year

In [0]:
ccu013_02_prim_care_meds_cohort_first = spark.sql('''
select m.Person_ID_DEID, 
  m.PrescribedBNFCode, 
  min(m.ProcessingPeriodDate) as first_prescrip_event_in_the_year, 
  m.age_band_23_01_2020, 
  m.sex, 
  m.BNF6  
from 
  dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort m 
group by m.Person_ID_DEID, m.PrescribedBNFCode, m.age_band_23_01_2020, m.sex, m.BNF6 ''')

In [0]:
ccu013_02_prim_care_meds_cohort_first = ccu013_02_prim_care_meds_cohort_first.withColumn("BNFcomp", ccu013_02_prim_care_meds_cohort_first.PrescribedBNFCode.substr(1,9))
ccu013_02_prim_care_meds_cohort_first.show()

In [0]:
ccu013_02_prim_care_meds_cohort_first = ccu013_02_prim_care_meds_cohort_first.withColumn("BNF2", ccu013_02_prim_care_meds_cohort_first.PrescribedBNFCode.substr(1,2))
ccu013_02_prim_care_meds_cohort_first.show()

In [0]:
ccu013_02_prim_care_meds_cohort_first.createOrReplaceGlobalTempView('ccu013_02_prim_care_meds_cohort_first') 

In [0]:
drop_table("ccu013_02_prim_care_meds_cohort_first")

In [0]:
create_table("ccu013_02_prim_care_meds_cohort_first")

In [0]:
%sql
select * from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first limit 5;

In [0]:
%sql 
select BNF6, count(distinct(Person_ID_DEID)) 
from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first where BNF6= ''
group by BNF6;

In [0]:
%sql 
select count(distinct(BNF6)) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first;

In [0]:
%sql 
select count(distinct(PrescribedBNFCode)) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first;

In [0]:
%sql 
select count(distinct(person_ID_DEID)) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first;

In [0]:
%sql 
select count(distinct(person_ID_DEID)) from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t;

In [0]:
%sql 
select count(*) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first;

In [0]:
%sql 
select count(*) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort;

In [0]:
%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort;

In [0]:
%sql 
select count(distinct(Person_ID_DEID)) as n_indiv, age_band_23_01_2020, sex from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort 
group by BNF6, age_band_23_01_2020, sex
order by BNF6, age_band_23_01_2020, sex;

In [0]:
ccu013_02_prim_care_meds_cohort_bands = spark.sql('''
select BNF6, age_band_23_01_2020, sex, count(distinct(Person_ID_DEID)) as n_indiv  from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort 
group by BNF6, age_band_23_01_2020, sex
order by BNF6, age_band_23_01_2020, sex''')

In [0]:
ccu013_02_prim_care_meds_cohort_bands.createOrReplaceGlobalTempView('ccu013_02_prim_care_meds_cohort_bands') 

In [0]:
drop_table("ccu013_02_prim_care_meds_cohort_bands")

In [0]:
create_table("ccu013_02_prim_care_meds_cohort_bands")

In [0]:
%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_bands limit 26;

In [0]:
%sql 
select count(*) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_bands;

In [0]:
%sql 
select count(distinct(BNF6)) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_bands;

In [0]:
%sql 
select count(*) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first WHERE PrescribedBNFCode IS NULL;

In [0]:
%sql 
select distinct(BNF6) from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first
order by BNF6;

Counts of n individual in each bnf paragraph per age and sex band (for R) per BNFcomp

In [0]:
ccu013_02_prim_care_meds_cohort_bands_comp = spark.sql('''
select BNFcomp, age_band_23_01_2020, sex, count(distinct(Person_ID_DEID)) as n_indiv  from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first 
group by BNFcomp, age_band_23_01_2020, sex
order by BNFcomp, age_band_23_01_2020, sex''')

In [0]:
ccu013_02_prim_care_meds_cohort_bands_comp.createOrReplaceGlobalTempView('ccu013_02_prim_care_meds_cohort_bands_comp') 

In [0]:
drop_table("ccu013_02_prim_care_meds_cohort_bands_comp")

In [0]:
create_table("ccu013_02_prim_care_meds_cohort_bands_comp")

In [0]:
%sql 
select BNFcomp, age_band_23_01_2020, sex, count(distinct(Person_ID_DEID)) as n_indiv  from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first 
group by BNFcomp, age_band_23_01_2020, sex
order by BNFcomp, age_band_23_01_2020, sex;

In [0]:
%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_bands_comp limit 20;

In [0]:
%sql 
select count(distinct(BNFcomp)) as n_unique_BNFcomp from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first 
where Person_ID_Deid = ''
and BNF2 NOT in ('18', '19', '20', '21', '22', '23');

counts by compound

In [0]:
%sql 
select count(distinct(BNFcomp)) as n_unique_BNFcomp from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first;

In [0]:
%sql 
select BNFcomp, count(distinct(Person_ID_DEID)) as n_indiv from dars_nic_391419_j3w9t_collab.ccu013_02_prim_care_meds_cohort_first 
group by BNFcomp
order by n_indiv desc;

In [0]:
%sql 
select * from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t 
where PrescribedBNFCode='';

In [0]:
%sql 
select * from dars_nic_391419_j3w9t.primary_care_meds_dars_nic_391419_j3w9t where PrescribedBNFCode is NULL;